# crash course on the dataslicer

Here we illustrate how the dataslicer package can be used to analyze ZTF catalogs (PSF catalogs in this case, although sextractor ones can also be handled by the code). In particular, we will perform all the heavy-lifting pre-processing steps needed to investigate the photometry of all the sources in one of the deep drilling fields (`fieldid == 679`). These steps includes:

- joining metadata to the source tables. Both fits files header keywords and metadata from the IRSA archieve can be added. In the latter case, `ztfquery` needs to be installed (ask Mickael for permission: https://github.com/MickaelRigault/ztfquery/tree/master/ztfquery)

- preselect sources.

- group together the observations of the same sources through clustering of their sky coordinates.

- assign to each source the PS1 star from the PS1 calibrator (PS1Cal)sources database (see XXX for instruction on how to set up this databse)

- apply photometric solution provided by the IPAC pipeline

- outlier rejection based on comparison with the PS1Cal sources.


For testing, we have a few catalogs in `../deep_drilling/testdata/`.

## Loading and preselecting the data

we create a new dataset conatining only the data for a given readout-channel (# 43) and point it to the data directory.

with the `load` command the data is actually read. At first, only the fits file headers (for extension 0, as specified by the argument `metadata_ext`) are parsed into the `ds.metadata` DataFrame. The metadata are then used to select only the files matching a given criteria with the `expr` argument (in this case we want just one RC).
The source table is saved in `ds.objtable.df`, a `srcdf` (cource-dataframe) object. This is a subclass of `pandas.DataFrame` with some special methods added. 

To avoid re-reading the files eveytime the code is run, the dataframe are saved into csv files in the datadir. Unless you request different metadata columns, the fits files will not be touched anymore after the first time. Use `force_reload=True` to re-read from scratch.


Many `dataset` methods can produce diagnostic plots. With the `set_plot_dir` we specify where this plots have to be saved.

Finally we apply some pre-selection of the sources based on analysis flags and signal to noise and add the corresponding metdata to the `ds.objtable.df`.

In [1]:
%matplotlib inline

from dataslicer.dataset import dataset

# create dataset
ds = dataset("dstest_rc43", "../deep_drilling/testdata/")

# load the data for the corresponding RC and read the following keys from the headers
meta_keys = ['EXPID', 'RCID', 'MAGZP', 'MAGZPUNC', 'CLRCOEFF', 'CLRCOUNC', 'OBSMJD', 'FIELDID', 'FILTERID']
ds.load(metadata_ext = 0, objtable_ext = 'PSF_CATALOG',
        header_keys = meta_keys, 
        force_reload = True,
        expr = 'RCID == 43',
        downcast = False     # experimental: use this option to save memory,
       )

# set directory for diag plots
ds.set_plot_dir("./diag_plots")

# preselecyion
ds.select_objects("flags == 0 and snr>2")
    
# add meta to sources (metadata_cols can be a subset of meta_keys)
ds.merge_metadata_to_sources( metadata_cols = meta_keys, join_on = 'OBSID') 

# see what's there
print (ds.objtable.df.head(5))

INFO:dataslicer.dataset:found 7 .fits files in directory: ../deep_drilling/testdata/
INFO:dataslicer.dataset:Reading headers for metadata..
100%|██████████| 7/7 [00:00<00:00, 116.78it/s]
INFO:dataslicer.dataset:loaded meta data from fits headers for 7 files into metadata dataframe.
INFO:dataslicer.dataset:saving metadata dataframe to csv file: ../deep_drilling/testdata/dstest_rc43_metadata.csv
INFO:dataslicer.dataset:quering dataframe with: RCID == 43. 2 rows survived
INFO:dataslicer.dataset:loading files into object table..
INFO:dataslicer.dataset:using target metadata to indetify the files.
100%|██████████| 2/2 [00:00<00:00, 43.31it/s]
INFO:dataslicer.dataset:loaded 2 files into a 4083 rows dataframe. Took 5.54e-02 sec
INFO:dataslicer.dataset:saving objtable dataframe to csv file: ../deep_drilling/testdata/dstest_rc43_objtable.csv
INFO:dataslicer.dataset:diagnostic plots will be saved to ./diag_plots
INFO:dataslicer.dataset:quering dataframe with: flags == 0 and snr>2. 4072 rows surv

     chi        dec  flags         flux    mag          ra  sharp     sigflux  \
0  0.816  34.316337      0  2474.704834 -8.484  242.541508  0.070  105.075150   
1  1.038  34.314076      0  7952.927734 -9.751  242.219749 -0.007  136.243454   
2  1.015  34.316288      0  9466.090820 -9.940  243.073745  0.000  138.625610   
3  1.133  34.316259      0   980.371521 -7.478  243.206592  0.116  100.408653   
4  0.700  34.313510      0  1121.159790 -7.624  242.384298  0.036   80.957901   

   sigmag        snr    ...           srcID     EXPID  RCID      MAGZP  \
0   0.046  23.549999    ...     40655421430  40655421    43  26.335712   
1   0.019  58.369999    ...     40655421431  40655421    43  26.335712   
2   0.016  68.290001    ...     40655421432  40655421    43  26.335712   
3   0.111   9.760000    ...     40655421433  40655421    43  26.335712   
4   0.078  13.850000    ...     40655421434  40655421    43  26.335712   

   MAGZPUNC  CLRCOEFF  CLRCOUNC        OBSMJD  FIELDID  FILTERID  
0

## identifying the sources and matching with PS1Cal

The clustering sued `sklearn.DBScan` and spherical geometry. The `cluster_size_arcsec` determines the maximum alowed scale for a group of sources to be assigned to the same cluster (here we use 3 arcseconds). We then remove all the clusters with less than 2 sources (if `purge_df = True`). 

The cluster centroinds are then used as seeds for catalog matching with the PS1Cal database. We use a search radius of 3 arcsec. `match_to_PS1cal` will add to `dataset.objtable.df` the PS1Cal columns for the matches and the distance of the ZTF source to the closest PS1Cal source (`dist2ps1` column). This is used to reject sources too far away from their PS1 counterpart. The `select_clusters` method will produce a diagnostic plot with the distribution of `dist2ps1` and return a dataframe with the sources that did not pass the selection.

In [2]:
# cluster sources using sky position
ds.objtable.cluster_sources(cluster_size_arcsec=3, min_samples=2, xname="ra", yname="dec", purge_df=True)

# match cluster centroid to PS1Cal sources
ds.objtable.match_to_PS1cal(rs_arcsec=3, xname="ra", yname="dec", use='clusters', plot=True)
    
# now remove cluster if they contain a source too far away from the PS1 cal
removed_sources = ds.objtable.select_clusters("dist2ps1<1.", plot_x='dist2ps1', log=True)

# now we see there are more columns
print (ds.objtable.df.head(5))
print (ds.objtable.df.columns.values)

INFO:dataslicer.dataset:running DBSCAN to cluster 4072 sources into individual objects
INFO:dataslicer.dataset:using ra, dec as coordinates and a radius of 3.00 arcsec
INFO:dataslicer.dataset:found 1516 clusters with maximum size of 3.00 arcsec and minimum number of entries: 2
INFO:dataslicer.dataset:matching objtable entries to PS1 calibrator stars
INFO:dataslicer.dataset:using ra, dec as coordinates and a search radius of 3.00 arcsec
INFO:dataslicer.dataset:using mongo client at localhost:27017
INFO:dataslicer.dataset:connected to collection srcs of database ps1cal.
INFO:dataslicer.dataset:found 240327150 documents in source collection srcs.
INFO:dataslicer.dataset:set HEALPIX partition of order 16 with key 'hpxid_16'. Nested: True, Indexed: True, Resolution ["]: 3.221
INFO:dataslicer.dataset:no 2d sphere key found in catalog ps1cal
INFO:dataslicer.dataset:source collection has the following indexes: _id, hpxid_16, field, rcid
INFO:dataslicer.dataset:setting default search method to 

     chi        dec  flags          flux     mag          ra  sharp  \
1  1.038  34.314076      0   7952.927734  -9.751  242.219749 -0.007   
2  1.015  34.316288      0   9466.090820  -9.940  243.073745  0.000   
3  1.133  34.316259      0    980.371521  -7.478  243.206592  0.116   
4  0.700  34.313510      0   1121.159790  -7.624  242.384298  0.036   
6  0.455  34.313966      0  30389.849609 -11.207  242.799656  0.020   

      sigflux  sigmag         snr    ...      dist2ps1  e_gmag  e_imag  \
1  136.243454   0.019   58.369999    ...      0.470006   0.004   0.001   
2  138.625610   0.016   68.290001    ...      0.480078   0.006   0.002   
3  100.408653   0.111    9.760000    ...      0.543840   0.018   0.006   
4   80.957901   0.078   13.850000    ...      0.491688   0.006   0.004   
6  215.523666   0.008  141.000000    ...      0.437936   0.003   0.002   

   e_rmag  e_zmag    gmag       imag      ra_ps1       rmag       zmag  
1   0.004   0.002  17.035  16.452000  242.219803  16.61

![diagnostic plot](./diag_plots/dstest_rc43_select_clusters_hist_dist2ps1.png)


## Photometric solution and outlier removal

We apply IPAC zero-point and color coeffient (we have the PS1 calibrator colors) to the sources and create a new column to the dataframe called `cal_mag`. Errors on all the parameters is propagated and the resulting error is saved to `err_cal_mag`. 

`ps1based_outlier_rm_iqr` removes clusters based on the normalized difference between the cluster average magnitude and that of the associated PS1 calibrator star. The algorithm works as follows:
                
- the average magnitude is computed for each cluster and compared to the PS1 magnitude for the cluster counterpart.
                
- the dataset is divided in 'natural' magnitude bins using the Jenks algorythm.
                
- for each bin the inter quantile range (IQR) and the median of the (cluster - PS1) magnitude difference is computed.
                
- the (source - PS1) magnitude distance is computed and divided by the IQR. That's the quantity you cut on.

When the dataset contains observation in multiple filters, the `FILTERID` column (from the fits file headers) is used to select the right PS1 magnitudes. To facilitate comparison we create a new column called `ps1mag_band` wich can be used in a 1:1 comparison with `cal_mag`, without having to worry about which filter is being used. 

In [3]:
# apply IPAC photometric calibration
ds.objtable.calmag(
    "mag", err_mag_col="sigmag", calmag_col='cal_mag', filterid_col='FILTERID', clrcoeff_name='CLRCOEFF')
    
# cut based on PS1
ds.objtable.ps1based_outlier_rm_iqr('cal_mag', 2.5, filterid_col='FILTERID', n_mag_bins=10, plot=True)
    
# add band-wise PS1 magnitude column
ds.objtable.add_bandwise_PS1mag_for_filter('ps1mag_band', filterid_col='FILTERID')
    
# remove sources belonging to cluster with outliers
rejected = ds.objtable.select_clusters("abs(cal_mag - ps1mag_band) < 0.5", plot_x='cal_mag', plot_y='ps1mag_band')

INFO:dataslicer.dataset:using FILTERID to apply the right PS1 colors depending on the ZTF band.
INFO:dataslicer.dataset:Applying photometric calibration.
INFO:dataslicer.dataset:saving plot to ./diag_plots/dstest_rc43_calmag.png
INFO:dataslicer.dataset:rejecting outliers based on the IQR normalized magnitude difference wrt PS1 cal
INFO:dataslicer.dataset:adding band-wise PS1 magnitude column aux_ps1mag to the df depending on FILTERID
INFO:dataslicer.dataset:updating grouped dataframe
INFO:dataslicer.dataset:rejected 9 clusters based on IRQ normalized residuals. Took 4.76e-02 sec
INFO:dataslicer.dataset:saving plot to ./diag_plots/dstest_rc43_ps1based_outlier_rm.png
INFO:dataslicer.dataset:removing auxiliary column: aux_ps1mag
INFO:dataslicer.dataset:updating grouped dataframe
INFO:dataslicer.dataset:1696 sources and 848 clusters retained.
INFO:dataslicer.dataset:adding band-wise PS1 magnitude column ps1mag_band to the df depending on FILTERID
INFO:dataslicer.dataset:updating grouped da

![diagnostic plot](./diag_plots/dstest_rc43_ps1based_outlier_rm.png)
![diagnostic plot](./diag_plots/dstest_rc43_calmag.png)
![diagnostic plot](./diag_plots/dstest_rc43_select_clusters_scatter_cal_mag_vs_ps1mag_band.png)


## Retrieveing airmass from IRSA and saving results

Some important information such as the airmass is not written in the fits file header. We use the exposure IDs to query IRSA archieve and download the metatdata for our sources. 

In [4]:
# add airmass from IRSA to the files (in this case just the airmass)
ds.objtable.df = ds.objtable.df.add_IRSA_meta()
    
print (ds.objtable.df.head(2))
print (ds.objtable.df.columns.values)

INFO:dataslicer.srcdf:found 2 unique exposures (EXPID) in metadata.
INFO:dataslicer.srcdf:querying IRSA using: expid+IN+(40655421,41356262)
INFO:dataslicer.srcdf:retrieved 128 metadata
INFO:dataslicer.srcdf:selecting IRSA meta columns: airmass
INFO:dataslicer.srcdf:adding the following columns to metadata dataframe: airmass, expid
INFO:dataslicer.srcdf:joined IRSA meta to dataframe. The following columns are now available: chi, dec, flags, flux, mag, ra, sharp, sigflux, sigmag, snr, sourceid, xpos, ypos, OBSID, srcID, EXPID, RCID, MAGZP, MAGZPUNC, CLRCOEFF, CLRCOUNC, OBSMJD, FIELDID, FILTERID, clusterID, dec_ps1, dist2ps1, e_gmag, e_imag, e_rmag, e_zmag, gmag, imag, ra_ps1, rmag, zmag, cal_mag, err_cal_mag, aux_ps1mag, norm_mag_dist, ps1mag_band, airmass


     chi        dec  flags         flux    mag          ra  sharp     sigflux  \
0  1.038  34.314076      0  7952.927734 -9.751  242.219749 -0.007  136.243454   
1  1.038  34.314076      0  7952.927734 -9.751  242.219749 -0.007  136.243454   

   sigmag        snr   ...       imag      ra_ps1    rmag       zmag  \
0   0.019  58.369999   ...     16.452  242.219803  16.618  16.412001   
1   0.019  58.369999   ...     16.452  242.219803  16.618  16.412001   

     cal_mag  err_cal_mag  aux_ps1mag  norm_mag_dist  ps1mag_band  airmass  
0  16.637059     0.019013      16.618       0.758151       16.618      1.0  
1  16.637059     0.019013      16.618       0.758151       16.618      1.0  

[2 rows x 42 columns]
['chi' 'dec' 'flags' 'flux' 'mag' 'ra' 'sharp' 'sigflux' 'sigmag' 'snr'
 'sourceid' 'xpos' 'ypos' 'OBSID' 'srcID' 'EXPID' 'RCID' 'MAGZP'
 'MAGZPUNC' 'CLRCOEFF' 'CLRCOUNC' 'OBSMJD' 'FIELDID' 'FILTERID'
 'clusterID' 'dec_ps1' 'dist2ps1' 'e_gmag' 'e_imag' 'e_rmag' 'e_zmag'
 'gmag' 'imag'

## saving results

For convenience, we add the band-wise magnitude difference to the dataframe and save it in a csv for later plotting and inspection.

In [5]:
# save the dataframe to file
import pandas as pd
to_save = pd.concat(
        [ds.objtable.df,
        ( ds.objtable.df['cal_mag'] - ds.objtable.df['ps1mag_band'] ).rename('mag_diff')],
        axis =1)
ds.objtable.df.to_csv("fuffa.csv", index=False)